In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

contradictory_my_dear_watson_path = kagglehub.competition_download('contradictory-my-dear-watson')

print('Data source import complete.')

Data source import complete.


Goal: To finetune a roBERTa classification model in order to identify the thing!

First, have to get into huggingface and nab a model.

In [ ]:
from huggingface_hub import login

login()

Ok with the data obtained, now lets actually get our hands dirty. First step, get the nec. libraries

In [ ]:
!pip install transformers datasets huggingface_hub tensorboard wandb evaluate
!pip install -U transformers
!pip install --upgrade accelerate
!pip install huggingface_hub optuna
!sudo apt-get install git-lfs --yes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 8.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    RobertaTokenizer,
    RobertaTokenizerFast,
    AutoModelForSequenceClassification,
    Trainer,
    TrainingArguments
)
torch.cuda.is_available()

True

In [ ]:
dataset = load_dataset(contradictory_my_dear_watson_path)

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from datasets import ClassLabel

# 1okenize function
def tokenize_function(example):
    return tokenizer(example['premise'],
                     example['hypothesis'],
                     truncation=True,
                     padding='max_length')

# tokenize all the splits splits
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# for each split, cast & rename
for split in tokenized_datasets:
    tokenized_datasets[split] = (
        tokenized_datasets[split]
        # convert float→int data + update schema
        .cast_column("label", ClassLabel(num_classes=3))
        # rename to the Trainer’s expected key
        .rename_column("label", "labels")
    )

# after cast/rename on train:
split = tokenized_datasets["train"].train_test_split(test_size=0.1, seed=42)
tokenized_datasets["train"] = split["train"]
tokenized_datasets["validation"] = split["test"]

# bind Torch format exactly once
tokenized_datasets.set_format(
    "torch",
    columns=["input_ids", "attention_mask", "labels"]
)

In [ ]:
import evaluate
import numpy as np

accuracy_metric = evaluate.load("accuracy")

# Define compute_metrics func
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=preds, references=labels)


In [ ]:
from transformers import (
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    EarlyStoppingCallback,
)
import optuna

model_checkpoint = "roberta-base"

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(
        model_checkpoint,
        num_labels=3
    )

# shared TrainingArguments
training_args = TrainingArguments(
    output_dir=f"./results/{model_checkpoint.replace('/', '_')}",
    eval_strategy    = "epoch",
    save_strategy          = "epoch",
    load_best_model_at_end = True,
    metric_for_best_model  = "eval_accuracy",
    greater_is_better      = True,
    save_total_limit       = 1,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size  = 16,
    num_train_epochs       = 3,
    logging_strategy       = "epoch",
)

# define the hyperparameter
def hp_space(trial):
    return {
        "learning_rate": trial.suggest_categorical("learning_rate", [1e-5, 2e-5]),
        "weight_decay":  trial.suggest_categorical("weight_decay",  [0.0, 0.01]),
    }

# init the trainer
trainer = Trainer(
    model_init      = model_init,
    args            = training_args,
    train_dataset   = tokenized_datasets['train'],
    eval_dataset    = tokenized_datasets['validation'],
    compute_metrics = compute_metrics,
    callbacks       = [EarlyStoppingCallback(early_stopping_patience=1)],
)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


74c9f5648fb38bc53a50b6fffe72f99dac81d881 <- api key

In [ ]:
# Run hyperparameter search
best_run = trainer.hyperparameter_search(
    direction="maximize",
    backend="optuna",
    hp_space=hp_space,
    n_trials=4,
    compute_objective=lambda metrics: metrics["eval_accuracy"],
)

print("✔︎ Best hyperparameters:", best_run.hyperparameters)

[I 2025-05-06 23:35:16,784] A new study created in memory with name: no-name-30642ef4-de6e-42e2-a5cf-9c5bfc59c112
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: bwoodsii (bwoodsii-stevens-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


...

eval/accuracy,▁██
eval/loss,█▁▁
eval/runtime,▃▁█
eval/samples_per_second,▆█▁
eval/steps_per_second,▆█▁
train/epoch,▁▁▅▅███
train/global_step,▁▁▅▅███
train/grad_norm,█▇▁
train/learning_rate,█▅▁
train/loss,█▄▁
eval/accuracy,0.68482


Epoch,Training Loss,Validation Loss,Accuracy
1,0.996100,0.825226,0.613861
2,0.766000,0.701643,0.669967
3,0.624100,0.724914,0.669967


[I 2025-05-07 00:20:02,218] Trial 3 finished with value: 0.66996699669967 and parameters: {'learning_rate': 2e-05, 'weight_decay': 0.0}. Best is trial 2 with value: 0.6848184818481848.


✔︎ Best hyperparameters: {'learning_rate': 2e-05, 'weight_decay': 0.01}


AttributeError: 'TrainingArguments' object has no attribute 'update'

In [ ]:
# update Trainer args with the best params found in the search
for name, value in best_run.hyperparameters.items():
    setattr(trainer.args, name, value)

trainer.train()

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,1.023500,0.836730,0.599010
2,0.785400,0.710983,0.674092
3,0.633300,0.729912,0.676568


TrainOutput(global_step=2046, training_loss=0.8140753096266343, metrics={'train_runtime': 663.1197, 'train_samples_per_second': 49.349, 'train_steps_per_second': 3.085, 'total_flos': 8610123481731072.0, 'train_loss': 0.8140753096266343, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.7299119830131531,
 'eval_accuracy': 0.6765676567656765,
 'eval_runtime': 7.5168,
 'eval_samples_per_second': 161.238,
 'eval_steps_per_second': 10.111,
 'epoch': 3.0}

Now its just a matter of saving the model!

In [ ]:
trainer.push_to_hub("Woodsii/BERT_base_kaggle_watson")

rng_state.pth:   0%|          | 0.00/14.2k [00:00<?, ?B/s]

Upload 28 LFS files:   0%|          | 0/28 [00:00<?, ?it/s]

optimizer.pt:   0%|          | 0.00/997M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

...

events.out.tfevents.1746576536.fbc1e4613f58.5808.3:   0%|          | 0.00/7.09k [00:00<?, ?B/s]

events.out.tfevents.1746577432.fbc1e4613f58.5808.4:   0%|          | 0.00/7.09k [00:00<?, ?B/s]

events.out.tfevents.1746578102.fbc1e4613f58.5808.5:   0%|          | 0.00/411 [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Woodsii/roberta-base/commit/1c88b0d6f7f530221ffa614322fb26f215e4504e', commit_message='Woodsii/BERT_base_kaggle_watson', commit_description='', oid='1c88b0d6f7f530221ffa614322fb26f215e4504e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Woodsii/roberta-base', endpoint='https://huggingface.co', repo_type='model', repo_id='Woodsii/roberta-base'), pr_revision=None, pr_num=None)